In [1]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
    
os.environ['HF_HOME'] = '/autodl-tmp/hf'

In [2]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

/root/miniconda3/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigbio/biored contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/biored
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Using the latest cached version of the module from /autodl-tmp/hf/modules/datasets_modules/datasets/bigbio--biored/18063bd2d68db98142aaf46df1a6436e066ae226b6d0fbe293bd452e7bcdce23 (last modified on Thu May 16 01:52:27 2024) since it couldn't be found locally at bigbio/biored, or remotely on the Hugging Face Hub.


In [3]:
import json
import os, sys
import contextlib
from text2knowledge.strategy1 import extract_concepts, extract_concepts_strict


model_name = 'mistral:latest'
i = 4
while i < len(datas['test']):
    data = datas['test'][i]
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }

    #with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    entities_extracted = extract_concepts_strict(abstract, model=model_name, metadata=metadata)
    
    if entities_extracted:
        disease_entities_extracted = [e for e in entities_extracted if e['category'] == 'Disease']
        # print(disease_entities_extracted)
        entities_str = json.dumps(entities_extracted, indent=4)
        # print(entities_str)
    else:
        print("No entities found.")
    break
    # i += 1
    # if i >= 2:
    #     break

 [{
"DGUOK": "<Gene name: DGUOK, type: Gene>",
"dGK": "<Protein name: dGK, type: Protein>"
},
{
"Mitochondrial DNA depletion": "<Disease name: Mitochondrial DNA depletion, type: Disease>",
"hepatocerebral phenotype": "<Symptom or Disease: hepatocerebral phenotype, type: Symptom/Disease>"
},
{
"DGUOK gene": "<Gene name: DGUOK, type: Gene>",
"dGK kinase domain": "<MolecularFunction: dGK kinase domain, type: MolecularFunction>"
}]An exception occurred: "entity" not in json string output. You must use "###{key}###" to enclose the {key}.
Current invalid json format: {
"DGUOK": "<Gene name: DGUOK, type: Gene>",
"dGK": "<Protein name: dGK, type: Protein>"
},
{
"Mitochondrial DNA depletion": "<Disease name: Mitochondrial DNA depletion, type: Disease>",
"hepatocerebral phenotype": "<Symptom or Disease: hepatocerebral phenotype, type: Symptom/Disease>"
},
{
"DGUOK gene": "<Gene name: DGUOK, type: Gene>",
"dGK kinase domain": "<MolecularFunction: dGK kinase domain, type: MolecularFunction>"
}
 {


In [4]:
abstract

'Deoxyguanosine kinase (dGK) deficiency is a frequent cause of mitochondrial DNA depletion associated with a hepatocerebral phenotype. In this study, we describe a new splice site mutation in the DGUOK gene and the clinical, radiologic, and genetic features of these DGUOK patients. This new DGUOK homozygous mutation (c.444-62C>A) was identified in three patients from two North-African consanguineous families with combined respiratory chain deficiencies and mitochondrial DNA depletion in the liver. Brain MRIs are normal in DGUOK patients in the literature. Interestingly, we found subtentorial abnormal myelination and moderate hyperintensity in the bilateral pallidi in our patients. This new mutation creates a cryptic splice site in intron 3 (in position -62) and is predicted to result in a larger protein with an in-frame insertion of 20 amino acids. In silico analysis of the putative impact of the insertion shows serious clashes in protein conformation: this insertion disrupts the alpha